# import

In [1]:
import os
import sys
import json
import pandas as pd

In [2]:
from config import *

sys.path.append(os.path.normpath('../../src'))
from experiment import AGG
from common import count_false_alarm

# Select best parameters

In [3]:
def get_best_prms(result):
    if not isinstance(result, pd.DataFrame):
        return {}
    df = result.copy()
    df['total'] = df['mcc'] - FA_WEIGHT * df['false alarm']
    keys = df.index.names
    vals = df.sort_values('total').index[-1]
    if isinstance(vals, tuple):
        vals = list(vals)
    else:
        vals = [vals]
    return {k: v for k, v in zip(keys, vals)}

In [4]:
def check_prms(detector, dataset, detector_prms):
    if len(detector_prms) == 0:
        return {}
    agg = AGG(SAVE_DATA_DIR, dataset, detector)
    df = agg.results
    df = df[df['seed'] < n_tune].copy()
    df = count_false_alarm(df)
    if df.groupby(by=detector_prms).count()['seed'].std() != 0:
        print(dataset, detector, detector_prms)
    return df.groupby(by=detector_prms).mean()[['mcc', 'false alarm']]

## Random seed split
[seed < n_tune]: for tuning, 
[seed >= n_tune]: for evaluation

In [5]:
n_tune = len(DATA_PRMS['seed']) // 2

In [6]:
results = {}
best_prms = {}
for detector in ALL_DETECTORS:
    results[detector] = {}
    best_prms[detector] = {}
    detector_prms = [k for k in DETECTOR_PRMSS[detector].keys()]
    for dataset in DATASETS:
        results[detector][dataset] = check_prms(detector, dataset, detector_prms)
        best_prms[detector][dataset] = get_best_prms(results[detector][dataset])

In [7]:
# from https://github.com/hmallen/numpyencoder/blob/master/numpyencoder/numpyencoder.py
class NumpyEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):

            return int(obj)

        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)
        
        elif isinstance(obj, (np.complex_, np.complex64, np.complex128)):
            return {'real': obj.real, 'imag': obj.imag}
        
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
    
        elif isinstance(obj, (np.bool_)):
            return bool(obj)

        elif isinstance(obj, (np.void)): 
            return None

        return json.JSONEncoder.default(self, obj)

In [8]:
with open('detector.conf', 'w') as f:
    json.dump(best_prms, f, indent=2, cls=NumpyEncoder)

for detector, tables in results.items():
    print('%s ====================================================' % detector)
    for dataset, table in tables.items():
        print(dataset)
        display(table.T)